<a href="https://colab.research.google.com/github/yashvoladoddi37/kanji-diffusion-yash/blob/main/kanji_finetune_training_and_inference_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies, logging into huggingface

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive")
!git clone https://github.com/huggingface/diffusers
!pip install git+https://github.com/huggingface/diffusers
os.chdir("diffusers/examples/text_to_image/")
!pip install -r requirements.txt
!huggingface-cli login

Training Script for finetuning Stable Diffusion 1.4, resulting in kanji-diffusion-v1-4

In [ ]:
!accelerate launch train_text_to_image_lora.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" \
  --dataset_name="yashvoladoddi37/kanjienglish"
  --caption_column="text" \
  --resolution=512 --random_flip \
  --train_batch_size=1 \
  --num_train_epochs=1 \
  --checkpointing_steps=500 \
  --learning_rate=1e-04 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --seed=42 \
  --output_dir="kanji-diffusion-v1-4" \
  --validation_prompt="A kanji meaning Elon Musk" \
  --push_to_hub

Training Script for finetuning Stable Diffusion 1.5, resulting in kanji-diffusion-v1-5

In [ ]:
!accelerate launch train_text_to_image_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --dataset_name="yashvoladoddi37/kanjienglish" \
  --image_column="image" \
  --caption_column="text" \
  --resolution=128 \
  --random_flip \
  --train_batch_size=1 \
  --num_train_epochs=1 \
  --checkpointing_steps=750 \
  --learning_rate=1e-04 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --seed=37 \
  --output_dir="kanji-diffusion-v1-5" \
  --validation_prompt="A kanji meaning Elon Musk" \
  --push_to_hub

Building a LoRa pipeline for finetuned model (version 1.4)

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_path = "yashvoladoddi37/kanji-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16, use_safetensors = True).to("cuda")
pipe.unet.load_attn_procs(model_path)
pipe.to("cuda")

prompt = "a Kanji meaning YouTube"
image = pipe(prompt).images[0]
image.save("youtube-kanji-v1-4.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Building a LoRa pipeline for finetuned model (version 1.5)

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_path = "yashvoladoddi37/kanji-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16, use_safetensors = True).to("cuda")
pipe.unet.load_attn_procs(model_path)
pipe.to("cuda")

prompt = "a Kanji meaning YouTube"
image = pipe(prompt).images[0]
image.save("youtube-kanji-v1-4.png")